In [1]:
# ✅ 1. Install Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# ✅ 2. Download Spark 3.5.0 from Apache Archive
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

# ✅ 3. Extract Spark
!tar -xzf spark-3.5.0-bin-hadoop3.tgz

# ✅ 4. Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

# ✅ 5. Install and initialize findspark
!pip install -q findspark
import findspark
findspark.init("/content/spark-3.5.0-bin-hadoop3")

# ✅ 6. Install Spark and Azure packages
!pip install -q pyspark==3.5.0
!pip install -q azure-eventhub fastavro

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init("/content/spark-3.5.0-bin-hadoop3")

In [3]:
!pip install fastavro confluent-kafka

# ***DRIVERS***

In [4]:
event_hub_namespace='iesstsabdbaa-grp-01-04'
driver_eventhub_conn_str = "Endpoint=sb://iesstsabdbaa-grp-01-04.servicebus.windows.net/;SharedAccessKeyName=driver_clean_policy;SharedAccessKey=8Nfg8SfybBuvg2bLHXl3nKPGM2IXn0JWT+AEhCk2Vgw=;EntityPath=g4_driver_clean_eh"
driver_eventhub_name = "g4_driver_clean_eh"

# Azure Blob
account_name='iesstsabdbaa'
account_key='yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g=='
account_key='yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g=='

container_name='g4-stream-output'

In [5]:
%%writefile avro_driver.py
#!/usr/bin/env python3

from confluent_kafka import Producer
import fastavro
import io
import sys
import time
import random
import os
import uuid

# ================= Schema =================
driver_schema = {
    "namespace": "ridehailing.driver",
    "type": "record",
    "name": "DriverEvent",
    "fields": [
        {"name": "driver_id", "type": "string"},
        {"name": "gender", "type": ["null", "string"], "default": None},
        {"name": "age", "type": ["null", "int"], "default": None},
        {"name": "rating", "type": ["null", "float"], "default": None},
        {"name": "ride_id", "type": ["null", "string"], "default": None},
        {"name": "timestamp", "type": "long"},
        {"name": "vehicle", "type": {
            "type": "enum",
            "name": "VehicleType",
            "symbols": ["STANDARD", "SHARED", "VAN", "PREMIUM"]
        }},
        {"name": "location", "type": {
            "type": "record",
            "name": "Location",
            "fields": [
                {"name": "latitude", "type": "double"},
                {"name": "longitude", "type": "double"}
            ]
        }},
        {"name": "status", "type": {
            "type": "enum",
            "name": "DriverStatus",
            "symbols": ["AVAILABLE", "ACCEPTED", "ONGOING", "COMPLETED"]
        }}
    ]
}
parsed_schema = fastavro.parse_schema(driver_schema)

# ================= Generator =================
def generate_driver_event():
    return {
        "driver_id": f"driver_{random.randint(1000, 9999)}",
        "gender": random.choices(["Male", "Female", None], weights=[0.40, 0.40, 0.20])[0],  # 10% chance of None
        "age": random.choice([None] + list(range(18, 75))),  # keep as is
        "rating": random.choice([round(random.uniform(1.0, 5.0), 1)] * 9 + [None]),  # 10% None
        "ride_id": str(uuid.uuid4()) if random.random() > 0.1 else None,
        "timestamp": int(time.time() * 1000),
        "vehicle": random.choice(["STANDARD", "SHARED", "VAN", "PREMIUM"]),
        "location": {
            "latitude": round(random.uniform(-90.0, 90.0), 6),
            "longitude": round(random.uniform(-180.0, 180.0), 6)
        },
        "status": random.choice(["AVAILABLE", "ACCEPTED", "ONGOING", "COMPLETED"]),
    }

# ================= Serializer =================
def avro_serialize(message):
    with io.BytesIO() as buf:
        fastavro.schemaless_writer(buf, parsed_schema, message)
        return buf.getvalue()

# ================= Args + Config =================
print(sys.argv)
if len(sys.argv) < 4:
    print("Usage: python avro_driver_producer.py <event_hub_namespace> <eventhub_name> <eventhub_connection_string>")
    sys.exit(1)

event_hub_namespace = sys.argv[1]
eventhub_name = sys.argv[2]
eventhub_connection_string = sys.argv[3]

conf = {
    'bootstrap.servers': f"{event_hub_namespace}.servicebus.windows.net:9093",
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': '$ConnectionString',
    'sasl.password': eventhub_connection_string,
    'client.id': 'driver-producer'
}

producer = Producer(**conf)

def delivery_report(err, msg):
    if err:
        print("❌ Delivery failed:", err)
    else:
        print(f"✅ Delivered to {msg.topic()} offset {msg.offset()}")

# ================= Streaming Loop =================
while True:
    record = generate_driver_event()
    avro_bytes = avro_serialize(record)
    print(record)
    producer.produce(topic=eventhub_name, value=avro_bytes, callback=delivery_report)
    producer.poll(0)
    time.sleep(0.01)

producer.flush()

Overwriting avro_driver.py


In [6]:
!nohup python3 avro_driver.py iesstsabdbaa-grp-01-04 g4_driver_clean_eh "Endpoint=sb://iesstsabdbaa-grp-01-04.servicebus.windows.net/;SharedAccessKeyName=driver_clean_policy;SharedAccessKey=8Nfg8SfybBuvg2bLHXl3nKPGM2IXn0JWT+AEhCk2Vgw=;EntityPath=g4_driver_clean_eh" > avro_driver_clean.log &

nohup: redirecting stderr to stdout


In [7]:
!sleep 10
!tail -20 avro_driver_clean.log

{'driver_id': 'driver_5536', 'gender': 'Female', 'age': 42, 'rating': 2.9, 'ride_id': 'cfb19539-5aeb-4df3-8cb2-a6729eb4cab9', 'timestamp': 1745420508522, 'vehicle': 'PREMIUM', 'location': {'latitude': 10.213359, 'longitude': -75.405805}, 'status': 'COMPLETED'}
✅ Delivered to g4_driver_clean_eh offset 5103515
✅ Delivered to g4_driver_clean_eh offset 5103517
{'driver_id': 'driver_8704', 'gender': 'Male', 'age': 22, 'rating': 2.2, 'ride_id': None, 'timestamp': 1745420508534, 'vehicle': 'PREMIUM', 'location': {'latitude': -7.926609, 'longitude': -85.749443}, 'status': 'ACCEPTED'}
{'driver_id': 'driver_9061', 'gender': 'Male', 'age': 31, 'rating': 3.2, 'ride_id': '8a867cd9-799d-4653-85d5-b19c3a190745', 'timestamp': 1745420508544, 'vehicle': 'STANDARD', 'location': {'latitude': -53.861768, 'longitude': -152.894218}, 'status': 'COMPLETED'}
✅ Delivered to g4_driver_clean_eh offset 5103520
✅ Delivered to g4_driver_clean_eh offset 5103523
{'driver_id': 'driver_8391', 'gender': 'Female', 'age': 3

In [8]:
import os
import subprocess

# Fetch the latest Spark 3.x.x version
# curl -s https://downloads.apache.org/spark/ → Fetches the Spark download page.
# grep -o 'spark-3\.[0-9]\+\.[0-9]\+' → Extracts only versions that start with spark-3. (ignoring Spark 4.x if it exists in the future).
# sort -V → Sorts the versions numerically.
# tail -1 → Selects the latest version.
spark_version = "spark-3.5.0"

spark_version

'spark-3.5.0'

In [9]:
spark_release=spark_version
hadoop_version='hadoop3'

import os, time
start=time.time()
os.environ['SPARK_RELEASE']=spark_release
os.environ['HADOOP_VERSION']=hadoop_version
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_release}-bin-{hadoop_version}"

In [10]:
# AVRO schema as a string (your driver schema)
driver_schema = """
{
  "namespace": "ridehailing.driver",
  "type": "record",
  "name": "DriverEvent",
  "fields": [
    {"name": "driver_id", "type": "string"},
    {"name": "gender", "type": ["null", "string"], "default": null},
    {"name": "age", "type": ["null", "int"], "default": null},
    {"name": "rating", "type": ["null", "float"], "default": null},
    {"name": "ride_id", "type": ["null", "string"], "default": null},
    {"name": "timestamp", "type": "long"},
    {"name": "vehicle", "type": {
        "type": "enum",
        "name": "VehicleType",
        "symbols": ["STANDARD", "SHARED", "VAN", "PREMIUM"]
    }},
    {
      "name": "location",
      "type": {
        "type": "record",
        "name": "Location",
        "fields": [
          {"name": "latitude", "type": "double"},
          {"name": "longitude", "type": "double"}
        ]
      }
    },
    {
      "name": "status",
      "type": {
        "type": "enum",
        "name": "DriverStatus",
        "symbols": ["AVAILABLE", "ACCEPTED", "ONGOING", "COMPLETED"]
      }
    }
  ]
}
"""

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro

eventhub_name = "g4_driver_clean_eh"
event_hub_namespace = "iesstsabdbaa-grp-01-04"
consumer_eventhub_connection_str = driver_eventhub_conn_str  # your key string

spark = SparkSession.builder \
    .appName("DriverStreamProcessor") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config("spark.sql.shuffle.partitions", 4) \
    .config("spark.jars.packages",
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,"
            "org.apache.spark:spark-avro_2.12:3.5.0,"
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .master("local[*]") \
    .getOrCreate()

In [12]:
kafkaConf_driver_clean = {
    "kafka.bootstrap.servers": f"{event_hub_namespace}.servicebus.windows.net:9093",
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{driver_eventhub_conn_str}";',
    "subscribe": "g4_driver_clean_eh",
    "startingOffsets": "latest",
    "groupIdPrefix": "driver_stream_clean_",
    "enable.auto.commit": "true",
    "auto.commit.interval.ms": "5000"
}

In [13]:
kafkaConf_driver_clean

{'kafka.bootstrap.servers': 'iesstsabdbaa-grp-01-04.servicebus.windows.net:9093',
 'kafka.sasl.mechanism': 'PLAIN',
 'kafka.security.protocol': 'SASL_SSL',
 'kafka.sasl.jaas.config': 'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://iesstsabdbaa-grp-01-04.servicebus.windows.net/;SharedAccessKeyName=driver_clean_policy;SharedAccessKey=8Nfg8SfybBuvg2bLHXl3nKPGM2IXn0JWT+AEhCk2Vgw=;EntityPath=g4_driver_clean_eh";',
 'subscribe': 'g4_driver_clean_eh',
 'startingOffsets': 'latest',
 'groupIdPrefix': 'driver_stream_clean_',
 'enable.auto.commit': 'true',
 'auto.commit.interval.ms': '5000'}

In [14]:
from pyspark.sql.functions import col
from pyspark.sql.avro.functions import from_avro

# Step 1: Read from Event Hub (Kafka-compatible)
df = spark.readStream.format("kafka").options(**kafkaConf_driver_clean).load()

# Step 2: Deserialize Avro messages using permissive mode
deserialized_df = df.select(
    from_avro(col("value"), driver_schema, {"mode": "PERMISSIVE"}).alias("driver")
)

# Step 3: Print schema to inspect structure
deserialized_df.printSchema()

root
 |-- driver: struct (nullable = true)
 |    |-- driver_id: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- age: integer (nullable = true)
 |    |-- rating: float (nullable = true)
 |    |-- ride_id: string (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |    |-- vehicle: string (nullable = true)
 |    |-- location: struct (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |-- status: string (nullable = true)



In [15]:
# Step 3: Flatten the nested structure
driver_flat_df = deserialized_df.select(
    col("driver.driver_id"),
    col("driver.gender"),
    col("driver.age"),
    col("driver.rating"),
    col("driver.ride_id"),
    col("driver.timestamp"),
    col("driver.vehicle"),
    col("driver.location.latitude").alias("latitude"),
    col("driver.location.longitude").alias("longitude"),
    col("driver.status")
)

In [18]:
query_name = "driver_query_clean"

driver_query = driver_flat_df.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName(query_name) \
    .start()

IllegalArgumentException: Cannot start query with name driver_query_clean as a query with that name is already active in this SparkSession

In [20]:
import time
time.sleep(10)
spark.sql("SHOW TABLES").show()
spark.sql("SELECT COUNT(*) FROM driver_query_clean").show()
spark.sql("SELECT * FROM driver_query_clean").show(truncate=False)

+---------+------------------+-----------+
|namespace|         tableName|isTemporary|
+---------+------------------+-----------+
|         |driver_query_clean|       true|
+---------+------------------+-----------+

+--------+
|count(1)|
+--------+
|   11630|
+--------+

+-----------+------+---+------+------------------------------------+-------------+--------+----------+-----------+---------+
|driver_id  |gender|age|rating|ride_id                             |timestamp    |vehicle |latitude  |longitude  |status   |
+-----------+------+---+------+------------------------------------+-------------+--------+----------+-----------+---------+
|driver_8528|Female|50 |1.2   |61ced494-1984-4b18-a630-de3f30b1027d|1745420605167|VAN     |38.909043 |-131.538799|COMPLETED|
|driver_1510|Male  |41 |3.5   |1c94fab5-c5cc-4660-ad5f-ed9c3a2d41f8|1745420605173|SHARED  |-66.22946 |-97.979245 |COMPLETED|
|driver_6796|Male  |59 |NULL  |6fcf1f8f-25c7-4a52-bec4-61f14e6eadd2|1745420605173|STANDARD|-9.656881 |-

# ***PASSENGERS***

In [21]:
event_hub_namespace='iesstsabdbaa-grp-01-04'
passenger_eventhub_conn_str = "Endpoint=sb://iesstsabdbaa-grp-01-04.servicebus.windows.net/;SharedAccessKeyName=passenger_clean_policy;SharedAccessKey=Up0Ivj1ZVQzMluCQ8MeWAXMFH5o53xBiM+AEhBT3ZG8=;EntityPath=g4_passenger_clean_eh"
passenger_eventhub_name = "g4_passenger_clean_eh"

# Azure Blob
account_name='iesstsabdbaa'
account_key='yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g=='
account_key='yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g=='

container_name='g4-stream-output'

In [22]:
%%writefile avro_passenger.py
#!/usr/bin/env python3

from confluent_kafka import Producer
import fastavro
import io
import sys
import time
import random
import os
import uuid

# ================= Schema =================
passenger_schema = {
    "namespace": "ridehailing.passenger",
    "type": "record",
    "name": "PassengerEvent",
    "fields": [
        {"name": "ride_id", "type": "string"},
        {"name": "passenger_id", "type": "string"},
        {"name": "passenger_name", "type": "string"},
        {"name": "gender", "type": ["null", "string"], "default": None},
        {"name": "age", "type": ["null", "int"], "default": None},
        {"name": "timestamp", "type": "long"},
        {
            "name": "pickup_location",
            "type": {
                "type": "record",
                "name": "pickupLocation",
                "fields": [
                    {"name": "latitude", "type": "double"},
                    {"name": "longitude", "type": "double"}
                ]
            }
        },
        {
            "name": "dropoff_location",
            "type": {
                "type": "record",
                "name": "dropoffLocation",
                "fields": [
                    {"name": "latitude", "type": "double"},
                    {"name": "longitude", "type": "double"}
                ]
            }
        },
        {
            "name": "event_type",
            "type": {
                "type": "enum",
                "name": "RequestType",
                "symbols": ["REQUEST", "CANCELLATION"]
            }
         },
         {
            "name": "vehicle_type",
            "type": {
                "type": "enum",
                "name": "vehicleType",
                "symbols": ["STANDARD", "SHARED", "VAN", "PREMIUM"]
            }
        }
    ]
}


parsed_schema_passenger = fastavro.parse_schema(passenger_schema)

# ================= Generator =================
def generate_passenger_event():
    return {
        "ride_id": str(uuid.uuid4()),
        "passenger_id": f"passenger_{random.randint(1000, 9999)}",
        "passenger_name": random.choice(["Carlos", "Fatima", "Leo", "Aisha", "Zara"]),
        "gender": random.choice(["Male", "Female", None]),
        "age": random.choice([None] + list(range(18, 81))),
        "timestamp": int(time.time() * 1000),
        "pickup_location": {
            "latitude": round(random.uniform(-90.0, 90.0), 6),
            "longitude": round(random.uniform(-180.0, 180.0), 6)
        },
        "dropoff_location": {
            "latitude": round(random.uniform(-90.0, 90.0), 6),
            "longitude": round(random.uniform(-180.0, 180.0), 6)
        },
        "event_type": random.choice(["REQUEST", "CANCELLATION"]),
        "vehicle_type": random.choice(["STANDARD", "SHARED", "VAN", "PREMIUM"])
    }

# ================= Serializer =================
def avro_serialize(message):
    with io.BytesIO() as buf:
        fastavro.schemaless_writer(buf, parsed_schema_passenger, message)
        return buf.getvalue()

# ================= Args + Config =================
print(sys.argv)
if len(sys.argv) < 4:
    print("Usage: python avro_passenger.py <event_hub_namespace> <eventhub_name> <eventhub_connection_string>")
    sys.exit(1)

event_hub_namespace = sys.argv[1]
eventhub_name = sys.argv[2]
eventhub_connection_string = sys.argv[3]

conf = {
    'bootstrap.servers': f"{event_hub_namespace}.servicebus.windows.net:9093",
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': '$ConnectionString',
    'sasl.password': eventhub_connection_string,
    'client.id': 'driver-producer'
}

producer = Producer(**conf)

def delivery_report(err, msg):
    if err:
        print("❌ Delivery failed:", err)
    else:
        print(f"✅ Delivered to {msg.topic()} offset {msg.offset()}")

# ================= Streaming Loop =================
while True:
    record = generate_passenger_event()
    avro_bytes = avro_serialize(record)
    print(record)
    producer.produce(topic=eventhub_name, value=avro_bytes, callback=delivery_report)
    producer.poll(0)
    time.sleep(0.01)

producer.flush()

Overwriting avro_passenger.py


In [23]:
!nohup python3 avro_passenger.py iesstsabdbaa-grp-01-04 g4_passenger_clean_eh "Endpoint=sb://iesstsabdbaa-grp-01-04.servicebus.windows.net/;SharedAccessKeyName=passenger_clean_policy;SharedAccessKey=Up0Ivj1ZVQzMluCQ8MeWAXMFH5o53xBiM+AEhBT3ZG8=;EntityPath=g4_passenger_clean_eh" > avro_passenger_clean.log &

nohup: redirecting stderr to stdout


In [24]:
!sleep 10
!tail -20 avro_passenger_clean.log

{'ride_id': 'b5c9a090-55b6-40e1-8ffd-01178cf29bdc', 'passenger_id': 'passenger_3177', 'passenger_name': 'Fatima', 'gender': 'Female', 'age': 72, 'timestamp': 1745420684991, 'pickup_location': {'latitude': -67.384581, 'longitude': -130.737996}, 'dropoff_location': {'latitude': 89.75111, 'longitude': 42.187193}, 'event_type': 'CANCELLATION', 'vehicle_type': 'STANDARD'}
✅ Delivered to g4_passenger_clean_eh offset 2075081
{'ride_id': '0a3c75b5-2be0-4079-a9fe-b1c154777eb6', 'passenger_id': 'passenger_6505', 'passenger_name': 'Zara', 'gender': None, 'age': 31, 'timestamp': 1745420685002, 'pickup_location': {'latitude': -89.136287, 'longitude': -17.060761}, 'dropoff_location': {'latitude': 78.105817, 'longitude': -123.333635}, 'event_type': 'REQUEST', 'vehicle_type': 'STANDARD'}
{'ride_id': 'c89b1851-621e-412a-a940-a194ba3d7185', 'passenger_id': 'passenger_3455', 'passenger_name': 'Zara', 'gender': 'Male', 'age': 18, 'timestamp': 1745420685012, 'pickup_location': {'latitude': -20.248903, 'lon

In [25]:
# AVRO schema for passengers
passenger_avro_schema = """
{
    "namespace": "ridehailing.passenger",
    "type": "record",
    "name": "PassengerEvent",
    "fields": [
        {"name": "ride_id", "type": "string"},
        {"name": "passenger_id", "type": "string"},
        {"name": "passenger_name", "type": "string"},
        {"name": "gender", "type": ["null", "string"], "default": null},
        {"name": "age", "type": ["null", "int"], "default": null},
        {"name": "timestamp", "type": "long"},
        {
            "name": "pickup_location",
            "type": {
                "type": "record",
                "name": "pickupLocation",
                "fields": [
                    {"name": "latitude", "type": "double"},
                    {"name": "longitude", "type": "double"}
                ]
            }
        },
        {
            "name": "dropoff_location",
            "type": {
                "type": "record",
                "name": "dropoffLocation",
                "fields": [
                    {"name": "latitude", "type": "double"},
                    {"name": "longitude", "type": "double"}
                ]
            }
        },
        {
            "name": "event_type",
            "type": {
                "type": "enum",
                "name": "RequestType",
                "symbols": ["REQUEST", "CANCELLATION"]
            }
         },
         {
            "name": "vehicle_type",
            "type": {
                "type": "enum",
                "name": "vehicleType",
                "symbols": ["STANDARD", "SHARED", "VAN", "PREMIUM"]
            }
        }
    ]
}
"""

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro

eventhub_name = "g4_passenger_clean_eh"
event_hub_namespace = "iesstsabdbaa-grp-01-04"
consumer_eventhub_connection_str = passenger_eventhub_conn_str  # defined earlier

# Create Spark session
spark = SparkSession.builder \
    .appName("PassengerStreamProcessor") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-avro_2.12:3.5.0') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \
    .getOrCreate()

In [27]:
kafkaConf_passenger_clean = {
    "kafka.bootstrap.servers": f"{event_hub_namespace}.servicebus.windows.net:9093",
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{passenger_eventhub_conn_str}";',
    "subscribe": "g4_passenger_clean_eh",
    "startingOffsets": "latest",
    "groupIdPrefix": "passenger_stream_clean_",
    "enable.auto.commit": "true",
    "auto.commit.interval.ms": "5000"
}

In [28]:
kafkaConf_passenger_clean

{'kafka.bootstrap.servers': 'iesstsabdbaa-grp-01-04.servicebus.windows.net:9093',
 'kafka.sasl.mechanism': 'PLAIN',
 'kafka.security.protocol': 'SASL_SSL',
 'kafka.sasl.jaas.config': 'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://iesstsabdbaa-grp-01-04.servicebus.windows.net/;SharedAccessKeyName=passenger_clean_policy;SharedAccessKey=Up0Ivj1ZVQzMluCQ8MeWAXMFH5o53xBiM+AEhBT3ZG8=;EntityPath=g4_passenger_clean_eh";',
 'subscribe': 'g4_passenger_clean_eh',
 'startingOffsets': 'latest',
 'groupIdPrefix': 'passenger_stream_clean_',
 'enable.auto.commit': 'true',
 'auto.commit.interval.ms': '5000'}

In [29]:
from pyspark.sql.functions import col
from pyspark.sql.avro.functions import from_avro

# Step 1: Read from Event Hub (Kafka-compatible)
df_passenger = spark.readStream.format("kafka").options(**kafkaConf_passenger_clean).load()

# Step 2: Deserialize Avro messages using permissive mode
deserialized_df_passenger = df_passenger.select(
    from_avro(col("value"), passenger_avro_schema, {"mode": "PERMISSIVE"}).alias("passenger")
)

# Step 3: Print schema to inspect structure
deserialized_df_passenger.printSchema()

root
 |-- passenger: struct (nullable = true)
 |    |-- ride_id: string (nullable = true)
 |    |-- passenger_id: string (nullable = true)
 |    |-- passenger_name: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- age: integer (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |    |-- pickup_location: struct (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |-- dropoff_location: struct (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |-- event_type: string (nullable = true)
 |    |-- vehicle_type: string (nullable = true)



In [30]:
#Flatten the nested structure
from pyspark.sql.functions import col

passenger_flat_df = deserialized_df_passenger.select(
    col("passenger.ride_id"),
    col("passenger.passenger_id"),
    col("passenger.passenger_name"),
    col("passenger.gender"),
    col("passenger.age"),
    col("passenger.timestamp"),
    col("passenger.pickup_location.latitude").alias("pickup_latitude"),
    col("passenger.pickup_location.longitude").alias("pickup_longitude"),
    col("passenger.dropoff_location.latitude").alias("dropoff_latitude"),
    col("passenger.dropoff_location.longitude").alias("dropoff_longitude"),
    col("passenger.event_type"),
    col("passenger.vehicle_type")
)

In [31]:
for q in spark.streams.active:
    print(f"Query: {q.name}, IsActive: {q.isActive}, Status: {q.status['message']}")

Query: driver_query_clean, IsActive: True, Status: Processing new data


# ✅ Basic Analytics: Ride Counts & Status Distribution (Tumbling Window):

---



In [40]:
from pyspark.sql.functions import col, from_unixtime, window, count

# Step 1: Convert milliseconds to seconds, then to timestamp
driver_flat_df_fixed = driver_flat_df.withColumn(
    "event_time", (col("timestamp") / 1000).cast("timestamp")
)

# Step 2: Basic tumbling window analytics by driver status
basic_analytics = driver_flat_df_fixed \
    .withWatermark("event_time", "1 minute") \
    .groupBy(
        window(col("event_time"), "1 minute"),
        col("status")
    ) \
    .agg(count("*").alias("total_rides"))

# Step 3: Write the result to an in-memory table
query_name = "basic_analytics_table"

query = basic_analytics.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName(query_name) \
    .start()

# Step 4: Show available memory tables
spark.sql("SHOW TABLES").show()

IllegalArgumentException: Cannot start query with name basic_analytics_table as a query with that name is already active in this SparkSession

In [41]:
# Check the result table after letting it run a bit
spark.sql(f"SELECT * FROM {query_name}").show(truncate=False)

+------------------------------------------+---------+-----------+
|window                                    |status   |total_rides|
+------------------------------------------+---------+-----------+
|{2025-04-23 15:05:00, 2025-04-23 15:06:00}|AVAILABLE|57         |
|{2025-04-23 15:05:00, 2025-04-23 15:06:00}|ACCEPTED |57         |
|{2025-04-23 15:05:00, 2025-04-23 15:06:00}|ONGOING  |45         |
|{2025-04-23 15:05:00, 2025-04-23 15:06:00}|COMPLETED|46         |
+------------------------------------------+---------+-----------+



# ✅ Intermediate Analytics: Driver Availability Rate by Vehicle Type (Per Minute)

In [42]:
from pyspark.sql.functions import col, when, window, count, sum

# Step 1: Create event_time column
driver_with_time = driver_flat_df.withColumn("event_time", (col("timestamp") / 1000).cast("timestamp"))

# Step 2: Add a new column: 1 if available, 0 otherwise
driver_with_flag = driver_with_time.withColumn(
    "is_available", when(col("status") == "AVAILABLE", 1).otherwise(0)
)

# Step 3: Aggregate per minute per vehicle type
availability_stats = driver_with_flag \
    .withWatermark("event_time", "1 minute") \
    .groupBy(
        window(col("event_time"), "1 minute"),
        col("vehicle")
    ) \
    .agg(
        count("*").alias("total_seen"),
        sum("is_available").alias("available_count")
    ) \
    .withColumn("availability_rate", (col("available_count") / col("total_seen")))

# Step 4: Write to memory
query_name = "driver_availability_rate"
query_driver_availability = availability_stats.writeStream \
    .outputMode("update") \
    .format("memory") \
    .queryName(query_name) \
    .start()

In [44]:
import time
time.sleep(10)

spark.sql("SELECT * FROM driver_availability_rate").show(truncate=False)

+------------------------------------------+--------+----------+---------------+-------------------+
|window                                    |vehicle |total_seen|available_count|availability_rate  |
+------------------------------------------+--------+----------+---------------+-------------------+
|{2025-04-23 15:05:00, 2025-04-23 15:06:00}|VAN     |44        |14             |0.3181818181818182 |
|{2025-04-23 15:05:00, 2025-04-23 15:06:00}|SHARED  |37        |9              |0.24324324324324326|
|{2025-04-23 15:05:00, 2025-04-23 15:06:00}|STANDARD|64        |16             |0.25               |
|{2025-04-23 15:05:00, 2025-04-23 15:06:00}|PREMIUM |44        |12             |0.2727272727272727 |
+------------------------------------------+--------+----------+---------------+-------------------+



In [72]:
driver_availability_df = spark.sql("SELECT * FROM driver_availability_rate")
driver_availability_df.write.mode("overwrite").parquet("driver_availability.parquet")

# ✅ Advanced Analytics: Missing Critical Driver Fields (Anomaly Detection)

In [45]:
from pyspark.sql.functions import col

# Detect driver records with missing critical fields
driver_missing_fields_df = driver_flat_df \
    .withColumn("event_time", (col("timestamp") / 1000).cast("timestamp")) \
    .filter(
        col("gender").isNull() |
        col("rating").isNull() |
        col("age").isNull()
    )

# Stream it into memory for querying
query_name = "driver_missing_fields"
missing_fields_query = driver_missing_fields_df.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName(query_name) \
    .start()

In [47]:
import time
time.sleep(10)
spark.sql("SELECT * FROM driver_missing_fields").show(truncate=False)

+-----------+------+----+------+------------------------------------+-------------+--------+----------+-----------+---------+-----------------------+
|driver_id  |gender|age |rating|ride_id                             |timestamp    |vehicle |latitude  |longitude  |status   |event_time             |
+-----------+------+----+------+------------------------------------+-------------+--------+----------+-----------+---------+-----------------------+
|driver_9911|NULL  |53  |3.7   |445b94b3-a856-4798-b0ed-f4741fff7407|1745420834014|STANDARD|12.836505 |176.244596 |ACCEPTED |2025-04-23 15:07:14.014|
|driver_6135|Male  |40  |NULL  |8481acd6-8e19-421f-b557-8fa781c74cbb|1745420834025|VAN     |-67.256104|-34.865399 |COMPLETED|2025-04-23 15:07:14.025|
|driver_9849|NULL  |39  |2.9   |290d5c57-80d6-41e5-99f4-d0ea08657829|1745420834045|SHARED  |78.619421 |86.345729  |COMPLETED|2025-04-23 15:07:14.045|
|driver_3120|NULL  |24  |3.0   |ce3695c7-486f-4362-b115-540354310f50|1745420834046|SHARED  |62.27069

In [48]:
output_path_driver = "wasbs://g4-stream-output@iesstsabdbaa.blob.core.windows.net/driver-data"
output_path_passenger = "wasbs://g4-stream-output@iesstsabdbaa.blob.core.windows.net/passenger-data"

output_path_checkpoint_driver = "wasbs://g4-stream-output@iesstsabdbaa.blob.core.windows.net/checkpoints/driver"
output_path_checkpoint_passenger = "wasbs://g4-stream-output@iesstsabdbaa.blob.core.windows.net/checkpoints/passenger"

In [49]:
spark.conf.set(
    "fs.azure.account.key.iesstsabdbaa.blob.core.windows.net",
    "yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g=="
)

In [50]:
# Write driver data
driver_stream = driver_flat_df.writeStream \
    .format("parquet") \
    .option("path", output_path_driver) \
    .option("checkpointLocation", output_path_checkpoint_driver) \
    .outputMode("append") \
    .start()

# Write passenger data
passenger_stream = passenger_flat_df.writeStream \
    .format("parquet") \
    .option("path", output_path_passenger) \
    .option("checkpointLocation", output_path_checkpoint_passenger) \
    .outputMode("append") \
    .start()

In [52]:
driver_missing_df = spark.sql("SELECT * FROM driver_missing_fields")
driver_missing_df.write.mode("overwrite").parquet("driver_missing_fields.parquet")

In [54]:
!pip install -q streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


In [73]:
!ngrok config add-authtoken 2w8SxbawHjjZ4mvpFOG2U3PjJsR_7QENazfhENTgDwByJBECT

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [78]:
%%writefile dashboard.py
import streamlit as st
import pandas as pd

# Load Parquet files
df_missing_fields_streamlit = pd.read_parquet("driver_missing_fields.parquet")
df_availability = pd.read_parquet("driver_availability.parquet")

st.title("🚖 Driver Analytics Dashboard")

# Section 1: Missing Fields
st.subheader("⚠️ Drivers with Missing Critical Fields")

st.write("### Sample Data")
st.dataframe(df_missing_fields_streamlit)

st.write("### Missing Values Summary")
missing_counts = df_missing_fields_streamlit.isna().sum().reset_index()
missing_counts.columns = ['Field', 'Missing Count']
st.bar_chart(missing_counts.set_index('Field'))

st.write("### Filter by Vehicle Type")
vehicle_types = df_missing_fields_streamlit['vehicle'].dropna().unique().tolist()
selected = st.multiselect("Select vehicle types:", vehicle_types, default=vehicle_types)

filtered_df = df_missing_fields_streamlit[df_missing_fields_streamlit['vehicle'].isin(selected)]
st.write("Filtered Data", filtered_df)

# Section 2: Availability Rate
if df_availability is not None:
    st.subheader("🟢 Driver Availability Rate by Vehicle Type")

    if 'window' in df_availability.columns and isinstance(df_availability['window'][0], dict):
        df_availability['time_window'] = df_availability['window'].apply(
            lambda w: f"{w['start']} - {w['end']}"
        )
    else:
        df_availability['time_window'] = df_availability['window']

    st.write("### Line Chart: Availability Rate over Time")
    # Aggregate to fix pivot issue
    pivot_ready = df_availability.groupby(['time_window', 'vehicle'])['availability_rate'].mean().reset_index()
    pivot_chart = pivot_ready.pivot(index='time_window', columns='vehicle', values='availability_rate')
    st.line_chart(pivot_chart)


# Footer
st.markdown("---")
st.caption("🚀 Built with Streamlit")

Overwriting dashboard.py


In [79]:
from pyngrok import ngrok

ngrok.kill()  # Kill any previous sessions
public_url = ngrok.connect(8502)
print(f"🌐 Streamlit is live at: {public_url}")
!streamlit run dashboard.py &>/dev/null &

🌐 Streamlit is live at: NgrokTunnel: "https://b04b-34-80-112-55.ngrok-free.app" -> "http://localhost:8502"


In [ ]:
# Set to True and run cell when you want to stop your queries and Spark job.
if False:
  # Get the list of active streaming queries
  active_queries = spark.streams.active

# Print details about each active query
  for query in active_queries:
      query.stop()
      print(f"Query Name: {query.name}")
      print(f"Query ID: {query.id}")
      print(f"Query Status: {query.status}")
      print(f"Is Query Active: {query.isActive}")
      print("-" * 50)
  spark.stop()
  spark.sparkContext.stop()